<a href="https://colab.research.google.com/github/NickEsColR/MachineLearningV/blob/train/taller/Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Equipo**

- Nicolás Colmenares

- Carlos Martinez

**Situación:**
Una ciudad enfrenta un aumento significativo de casos de dengue, con una tasa de incidencia que supera el promedio nacional.
La anticipación de brotes es crucial para implementar medidas preventivas y reducir la propagación de la enfermedad.

**Objetivo:**
Desarrollar un modelo predictivo utilizando redes neuronales para pronosticar futuros brotes de dengue en cada barrio de la ciudad.
Utilizar una base de datos histórica de casos de dengue desde 2015 hasta 2022 para entrenar el modelo.
Anticiparse a los brotes con al menos 3 semanas de anticipación.

**Finalidad:**
Permitir a las autoridades de salud pública tomar acciones oportunas, como:
Preparar a las instituciones prestadoras de salud (IPS).
Gestionar recursos (carros fumigadores, limpieza de sumideros).
Capacitar a la comunidad.

1. Redes Neuronales Tradicinales (MLP)
2. Red Convolucional (CNN) adaptada a series temporales
3. Red Neuronal Recurrente (RNN) básica.
4. Modelo con LSTMs
5. Modelo con GRUs

## Diccionario

train.parquet - El conjunto de datos de entrenamiento
test.parquet - El conjunto de datos de prueba
sample_submission.csv - un ejemplo de un archivo a someter en la competencia

| **Variable**         | **Descripción**                                                                                      |
|-----------------------|------------------------------------------------------------------------------------------------------|
| id_bar               | identificador único del barrio                                                                      |
| anio                 | Año de ocurrencia                                                                                   |
| semana               | Semana de ocurrencia                                                                               |
| Estrato              | Estrato socioeconómico del barrio                                                                   |
| area_barrio          | Área del barrio en km²                                                                             |
| dengue               | Conteo de casos de dengue                                                                          |
| concentraciones      | Cantidad de visitas e intervención a lugares de concentración humana (Instituciones)                |
| vivienda             | Conteo de las visitas a viviendas a revisión y control de criaderos                                 |
| equipesado           | Conteo de las fumigaciones con Maquinaria Pesada                                                   |
| sumideros            | Conteo de las intervenciones a los sumideros                                                       |
| maquina              | Conteo de las fumigaciones con motomochila                                                         |
| lluvia_mean          | Lluvia promedio en la semana i                                                                     |
| lluvia_var           | Varianza de la lluvia en la semana i                                                               |
| lluvia_max           | Lluvia máxima en la semana i                                                                       |
| lluvia_min           | Lluvia mínima en la semana i                                                                       |
| temperatura_mean     | Temperatura promedio en la semana i                                                                |
| temperatura_var      | Varianza de la temperatura en la semana i                                                          |
| temperatura_max      | Temperatura máxima en la semana i                                                                  |
| temperatura_min      | Temperatura mínima en la semana i                                                                  |


# 0. Configuraciones de Colab

Mover Kaggle.json a la ubicación correcta después de subirlo

In [ ]:
#Estas líneas son comandos de shell que se ejecutan dentro del Jupyter notebook. Se usan para configurar las credenciales de la API de Kaggle, que son necesarias para descargar conjuntos de datos (datasets) desde Kaggle.

!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!rm -rf /content/kaggle/output
!rm -rf /content/kaggle/input

Descargar dataset de la competencia

In [ ]:
!kaggle competitions download -c aa-v-2025-i-pronosticos-nn-rnn-cnn

In [ ]:
!mkdir -p /content/kaggle/output
!mkdir -p /content/kaggle/input

In [ ]:
!mv aa-v-2025-i-pronosticos-nn-rnn-cnn.zip /content/kaggle/input

In [ ]:
!unzip /content/kaggle/input/aa-v-2025-i-pronosticos-nn-rnn-cnn.zip -d /content/kaggle/input/

In [ ]:
#/kaggle/input
import os
for dirname, _, filenames in os.walk('/content/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


# 1. Imports

In [ ]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, accuracy_score
from tqdm import tqdm
from datetime import datetime, timedelta # Importing the required modules datetime and timedelta

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
import copy

In [ ]:
#Printing library versions
print('Pandas:', pd.__version__)
print('Numpy:', np.__version__)
print('PyTorch:', torch.__version__)

In [ ]:
import warnings
warnings.filterwarnings("ignore")

#2. Configuración Inicial y Carga de Datos

In [ ]:
config = {
    "TRAIN_DIR": '/content/kaggle/input/df_train.parquet',
    "TEST_DIR": '/content/kaggle/input/df_test.parquet',
    "SUBMISSION_DIR": '/content/sample_submission.csv',
    "BATCH_SIZE": 32,
    "TARGET_COLUMN": 'dengue',
    "GROUP_COLUMN": 'id_bar',
    "WINDOW_SIZE": 5,
    "HORIZON": 3,
}

In [ ]:
# Configuración del dispositivo
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# Cargar datos
train_df = pd.read_parquet(config["TRAIN_DIR"])
test_df = pd.read_parquet(config["TEST_DIR"])

#3. Preprocesamiento de Datos

##3.1 Generar Columna fecha
Creamos la columna fecha basada en anio y semana, asignando el último día de cada semana como índice.

In [ ]:
# Generar columna 'fecha' (último día de la semana, domingo)
def last_day_of_week(year, week):
    first_day = datetime.strptime(f'{year} {week} 1', "%Y %W %w")
    days_ahead = 6 - first_day.weekday()
    last_day = first_day + timedelta(days=days_ahead)
    return last_day

train_df['fecha'] = train_df.apply(lambda row: last_day_of_week(row['anio'], row['semana']), axis=1)
test_df['fecha'] = test_df.apply(lambda row: last_day_of_week(row['anio'], row['semana']), axis=1)

In [ ]:
# Establecer 'fecha' como índice
train_df.set_index('fecha', inplace=True)
test_df.set_index('fecha', inplace=True)

In [ ]:
# Eliminar columnas innecesarias
#train_df.drop(['id_bar', 'anio', 'semana'], axis=1, inplace=True)
#test_df.drop(['id_bar', 'anio', 'semana'], axis=1, inplace=True)

Particionamos el dataset en entrenamiento hasta el año 2020 y validación el 2021

In [ ]:
# Dividir conjunto de entrenamiento en train y validation
train_df_full = train_df.copy()
train_df = train_df_full[train_df_full.index.year <= 2020].copy()
val_df = train_df_full[train_df_full.index.year >= 2021].copy()

##3.2 Selección de Características
Definimos las características de entrada, considerando las correlaciones altas entre variables (e.g., lluvia_mean y lluvia_var: 0.82). Para simplificar, usamos todas las características disponibles y dejamos que el modelo aprenda las relaciones.

In [ ]:
# Definir características (excluyendo variables altamente correlacionadas)
features = ['ESTRATO', 'area_barrio', 'concentraciones', 'vivienda', 'equipesado', 'sumideros', 'maquina',
            'lluvia_mean', 'temperatura_mean', 'temperatura_max']  # Selección basada en correlaciones
target = 'dengue'

##3.3 Normalización


Normalizamos las características y el objetivo usando StandardScaler. Identificamos las características numéricas (excluyendo id, id_bar y dengue):

Características: ESTRATO, area_barrio, concentraciones, vivienda, equipesado, sumideros, maquina, lluvia_mean, lluvia_var, lluvia_max, lluvia_min, temperatura_mean, temperatura_var, temperatura_max, temperatura_min. Ajustamos escaladores por separado para características y objetivo:

Se excluyeron variables como lluvia_var, lluvia_max, temperatura_var, y temperatura_min debido a sus altas correlaciones (e.g., lluvia_var y lluvia_mean: 0.82), para reducir redundancia y mejorar la estabilidad de los modelos.

In [ ]:
# Normalización
scaler_features = StandardScaler()
train_df[features] = scaler_features.fit_transform(train_df[features])
val_df[features] = scaler_features.transform(val_df[features])
test_df[features] = scaler_features.transform(test_df[features])

scaler_target = StandardScaler()
train_df[target] = scaler_target.fit_transform(train_df[[target]])
val_df[target] = scaler_target.transform(val_df[[target]])

##3.3 Crear Secuencias para Series Temporales
Para predecir con 3 semanas de anticipación, usamos una ventana de 5 semanas (window_size=5) y un horizonte de 3 semanas (horizon=3).

In [ ]:
def create_sequences(df, window_size, horizon, features, target, group_column):
    sequences = []
    labels = []
    groups = df.groupby(group_column)
    for _, group in groups:
        group = group.sort_index()
        for i in range(len(group) - window_size - horizon + 1):
            X = group.iloc[i:i + window_size][features].values
            y = group.iloc[i + window_size + horizon - 1][target]
            sequences.append(X)
            labels.append(y)
    return sequences, labels

train_sequences, train_labels = create_sequences(train_df, config["WINDOW_SIZE"], config["HORIZON"], features, target, config["GROUP_COLUMN"])
val_sequences, val_labels = create_sequences(val_df, config["WINDOW_SIZE"], config["HORIZON"], features, target, config["GROUP_COLUMN"])

##3.4 Dataset y DataLoader
Creamos un Dataset personalizado y dividimos en entrenamiento y validación.

In [ ]:
class DengueDataset(Dataset):
    def __init__(self, sequences, labels):
        self.sequences = sequences
        self.labels = labels

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        X = self.sequences[idx]
        y = self.labels[idx]
        return torch.tensor(X, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)

In [ ]:
class DengueTestDataset(Dataset):
    def __init__(self, sequences):
        self.sequences = sequences

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        X = self.sequences[idx]
        return torch.tensor(X, dtype=torch.float32)

In [ ]:
train_dataset = DengueDataset(train_sequences, train_labels)
val_dataset = DengueDataset(val_sequences, val_labels)

train_loader = DataLoader(train_dataset, batch_size=config["BATCH_SIZE"], shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=config["BATCH_SIZE"], shuffle=False)

#4. Implementación de Modelos

##4.1 Modelo MLP
Un Perceptrón Multicapa que aplana las secuencias.

In [ ]:
class MLPModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, dropout_rate=0.2):
        super(MLPModel, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_dim, output_dim)
        )

    def forward(self, x):
        batch_size = x.size(0)
        x = x.view(batch_size, -1)
        return self.model(x)

#4.2 Modelo CNN para Series Temporales
Una CNN 1D adaptada a series temporales.

In [ ]:
class CNNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, dropout_rate=0.2):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv1d(input_dim, hidden_dim, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv1d(hidden_dim, hidden_dim, kernel_size=3, padding=1)
        self.pool = nn.AdaptiveAvgPool1d(1)
        self.dropout = nn.Dropout(dropout_rate)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = x.permute(0, 2, 1)  # (batch, features, window_size)
        x = self.conv1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.pool(x).squeeze(-1) # La salida es (batch_size, hidden_dim)
        x = self.dropout(x)
        x = self.fc(x)
        return x

#4.3 Modelo RNN Básico
Implementación proporcionada con estados iniciales definidos.

In [ ]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class RNNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim, dropout_rate=0.2):
        super(RNNModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim
        self.rnn = nn.RNN(input_dim, hidden_dim, layer_dim, batch_first=True, nonlinearity='relu')
        self.dropout = nn.Dropout(dropout_rate)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).to(x.device)
        out, _ = self.rnn(x, h0)
        out = self.dropout(out[:, -1, :])
        out = self.fc(out)
        return out

#4.4 Modelo LSTM

In [ ]:
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim, dropout_rate=0.2):
        super(LSTMModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim
        self.lstm = nn.LSTM(input_dim, hidden_dim, layer_dim, batch_first=True)
        self.dropout = nn.Dropout(dropout_rate)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).to(x.device)
        c0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.dropout(out[:, -1, :])
        out = self.fc(out)
        return out

#4.5 Modelo GRU

In [ ]:
class GRUModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim, dropout_rate=0.2):
        super(GRUModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim
        self.gru = nn.GRU(input_dim, hidden_dim, layer_dim, batch_first=True)
        self.dropout = nn.Dropout(dropout_rate)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).to(x.device)
        out, _ = self.gru(x, h0)
        out = self.dropout(out[:, -1, :])
        out = self.fc(out)
        return out

#5. Entrenamiento y Evaluación

##5.1 Función de Entrenamiento

In [ ]:
def train_model(model, train_loader, val_loader, epochs, optimizer, criterion, device):
    model.to(device)
    train_losses = []
    val_losses = []
    for epoch in range(epochs):
        model.train()
        train_loss = 0
        for X, y in train_loader:
            X, y = X.to(device), y.to(device)
            optimizer.zero_grad()
            output = model(X)
            loss = criterion(output, y)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        train_loss /= len(train_loader)
        train_losses.append(train_loss)

        model.eval()
        val_loss = 0
        with torch.no_grad():
            for X, y in val_loader:
                X, y = X.to(device), y.to(device)
                output = model(X)
                loss = criterion(output, y)
                val_loss += loss.item()
        val_loss /= len(val_loader)
        val_losses.append(val_loss)

        if (epoch + 1) % 10 == 0:  # Imprimir solo cada 10 épocas
          print(f'Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')

    return train_losses, val_losses

##5.2 Función de Evaluación

In [ ]:
def evaluate_model(model, val_loader, device, scaler_target):
    model.eval()
    predictions = []
    actuals = []
    with torch.no_grad():
        for X, y in val_loader:
            X, y = X.to(device), y.to(device)
            output = model(X)
            predictions.append(output.cpu().numpy())
            actuals.append(y.cpu().numpy())
    predictions = np.concatenate(predictions)
    actuals = np.concatenate(actuals)
    predictions = scaler_target.inverse_transform(predictions.reshape(-1, 1)).flatten()
    actuals = scaler_target.inverse_transform(actuals.reshape(-1, 1)).flatten()
    mae = mean_absolute_error(actuals, predictions)
    mse = mean_squared_error(actuals, predictions)
    rmse = np.sqrt(mse)
    print(f'MAE: {mae:.4f}, MSE: {mse:.4f}, RMSE: {rmse:.4f}')
    return mae, mse, rmse, predictions, actuals

##5.3 Gráficos
Generamos gráficos de pérdidas y predicciones vs reales.

In [ ]:
def plot_losses(train_losses, val_losses):
    plt.plot(train_losses, label='Train Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('Training and Validation Losses')
    plt.show()

def plot_predictions(actuals, predictions):
    plt.plot(actuals, label='Actual')
    plt.plot(predictions, label='Predicted')
    plt.xlabel('Sample')
    plt.ylabel('Dengue Cases')
    plt.legend()
    plt.title('Actual vs Predicted Values')
    plt.show()

##5.4 Entrenar Modelos con optimización bayesiana

In [ ]:
def bayesian_optimization(model_class, train_dataset, val_dataset, space, max_evals=50, device='cpu'):
    """
    Realiza la optimización bayesiana de los hiperparámetros de un modelo.

    Args:
        model_class: La clase del modelo a optimizar.
        train_dataset: El conjunto de datos de entrenamiento.
        val_dataset: El conjunto de datos de validación.
        space: El espacio de búsqueda de hiperparámetros.
        max_evals: El número máximo de evaluaciones.
        device: El dispositivo a utilizar ('cpu' o 'cuda').

    Returns:
        Un diccionario con los mejores hiperparámetros y el mejor modelo.
    """

    def objective(params):
        """
        Función objetivo para la optimización bayesiana.
        """

        # Ajustar lógica si el modelo es CNN o MLP:
        if model_class == MLPModel:
            model = model_class(input_dim=config["WINDOW_SIZE"] * len(features),
                             hidden_dim=params['hidden_dim'], output_dim=1, dropout_rate=params['dropout_rate'])
        elif model_class == CNNModel:
            model = model_class(input_dim=len(features),
                             hidden_dim=params['hidden_dim'], output_dim=1, dropout_rate=params['dropout_rate'])
        else:
            model = model_class(input_dim=len(features),
                             hidden_dim=params['hidden_dim'], layer_dim=params['layer_dim'], output_dim=1, dropout_rate=params['dropout_rate'])

        criterion = nn.MSELoss()
        optimizer = params['optimizer'](model.parameters(), lr=params['lr'])
        train_losses, val_losses = train_model(model, train_dataset, val_dataset, epochs=params['epochs'], optimizer=optimizer, criterion=criterion, device=device)

        return {'loss': val_losses[-1], 'status': STATUS_OK, 'model': model}

    trials = Trials()
    best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=max_evals, trials=trials)

    # Get the best model from the trials
    best_trial = trials.best_trial
    best_model = best_trial['result']['model'] # Get the best model


    return {'best_params': best, 'best_model': best_model}

Hiperparametros del espacio de búsqueda

In [ ]:
epochs_choices = [50, 100, 150]
hidden_dim_choices = [32, 64, 128]
optimizer_choices = [optim.Adam, optim.RMSprop, optim.SGD]
layer_dim_choices = [1, 2, 3]

###5.4.1 MLP

In [ ]:
# # Define el espacio de búsqueda para MLP
# space_mlp = {
#     'epochs': hp.choice('epochs', epochs_choices),
#     'lr': hp.loguniform('lr', np.log(0.001), np.log(0.1)),
#     'optimizer': hp.choice('optimizer', optimizer_choices),
#     'hidden_dim': hp.choice('hidden_dim', hidden_dim_choices),
#     'dropout_rate': hp.uniform('dropout_rate', 0, 0.5)
# }

# # Realiza la optimización bayesiana para MLP
# best_model = bayesian_optimization(MLPModel, train_loader, val_loader, space_mlp, max_evals=50, device=DEVICE)

###5.4.1 CNN

In [ ]:
# # Define el espacio de búsqueda para CNN
# space_cnn = {
#     'epochs': hp.choice('epochs', epochs_choices),
#     'lr': hp.loguniform('lr', np.log(0.001), np.log(0.1)),
#     'optimizer': hp.choice('optimizer', optimizer_choices),
#     'hidden_dim': hp.choice('hidden_dim', hidden_dim_choices),
#     'dropout_rate': hp.uniform('dropout_rate', 0, 0.5)
# }

# # Realiza la optimización bayesiana para CNN
# best_model = bayesian_optimization(CNNModel, train_loader, val_loader, space_cnn, max_evals=50, device=DEVICE)

###5.4.1 RNN

In [ ]:
# # Define el espacio de búsqueda para RNN
# space_rnn = {
#     'epochs': hp.choice('epochs', epochs_choices),
#     'lr': hp.loguniform('lr', np.log(0.001), np.log(0.1)),
#     'optimizer': hp.choice('optimizer', optimizer_choices),
#     'hidden_dim': hp.choice('hidden_dim', hidden_dim_choices),
#     'layer_dim': hp.choice('layer_dim', layer_dim_choices),
#     'dropout_rate': hp.uniform('dropout_rate', 0, 0.5)
# }

# # Realiza la optimización bayesiana para RNN
# best_model = bayesian_optimization(RNNModel, train_loader, val_loader, space_rnn, max_evals=50, device=DEVICE)

###5.4.1 LSTM

In [ ]:
# # Define el espacio de búsqueda para LSTM
# space_lstm = {
#     'epochs': hp.choice('epochs', epochs_choices),
#     'lr': hp.loguniform('lr', np.log(0.001), np.log(0.1)),
#     'optimizer': hp.choice('optimizer', optimizer_choices),
#     'hidden_dim': hp.choice('hidden_dim', hidden_dim_choices),
#     'layer_dim': hp.choice('layer_dim', layer_dim_choices),  # Número de capas LSTM
#     'dropout_rate': hp.uniform('dropout_rate', 0, 0.5)
# }

# # Realiza la optimización bayesiana para LSTM
# best_model = bayesian_optimization(LSTMModel, train_loader, val_loader, space_lstm, max_evals=50, device=DEVICE)

###5.4.1 GRU

In [ ]:
# Define el espacio de búsqueda para GRU
space_gru = {
    'epochs': hp.choice('epochs', epochs_choices),
    'lr': hp.loguniform('lr', np.log(0.001), np.log(0.1)),
    'optimizer': hp.choice('optimizer', optimizer_choices),
    'hidden_dim': hp.choice('hidden_dim', hidden_dim_choices),
    'layer_dim': hp.choice('layer_dim', layer_dim_choices),  # Número de capas GRU
    'dropout_rate': hp.uniform('dropout_rate', 0, 0.5)
}

# Realiza la optimización bayesiana para GRU
best_model = bayesian_optimization(GRUModel, train_loader, val_loader, space_gru, max_evals=50, device=DEVICE)

## Mejores hiperpárametros

In [ ]:
# Imprime los mejores parámetros y el mejor modelo
print("Mejores parámetros:")
print(f"dropout_rate: {best_model['best_params']['dropout_rate']}")
print(f"epochs: {epochs_choices[best_model['best_params']['epochs']]}")
print(f"hidden_dim: {hidden_dim_choices[best_model['best_params']['hidden_dim']]}")
print(f"lr: {best_model['best_params']['lr']}")
print(f"optimizer: {optimizer_choices[best_model['best_params']['optimizer']]}")
if 'layer_dim' in best_model['best_params']:
  print(f"layer_dim: {layer_dim_choices[best_model['best_params']['layer_dim']]}")
print("Mejor modelo:", best_model['best_model'])

# 6. Predicción en el Test Set

##6.1 Crear Secuencias para Test
Combinamos train y test para obtener las semanas previas necesarias.

In [ ]:
combined_df = pd.concat([train_df.drop(columns=[target]), test_df], sort=False)
combined_df = combined_df.sort_values(by=['id_bar', 'fecha'])

test_sequences = []
ids = []

for idx, row in test_df.iterrows():
    id_bar = row['id_bar']
    fecha = row.name
    prev_dates = combined_df[(combined_df['id_bar'] == id_bar) & (combined_df.index < fecha)].tail(config["WINDOW_SIZE"])
    if len(prev_dates) == config["WINDOW_SIZE"]:
        seq = prev_dates[features].values
        test_sequences.append(seq)
        ids.append(row['id'])

test_sequences = np.array(test_sequences)
test_tensor = torch.tensor(test_sequences, dtype=torch.float32).to(DEVICE)

##6.2 Entrenar Modelo Final y Predecir

In [ ]:
best_model_trained = copy.deepcopy(best_model['best_model']).to(DEVICE)

In [ ]:
# 1. Preparar la data completa para el entrenamiento
train_df_full[features] = scaler_features.transform(train_df_full[features])
train_df_full[target] = scaler_target.transform(train_df_full[[target]])

# 2. Preparar la secuencia de la data completa
train_sequences_full, train_labels_full = create_sequences(train_df_full, config["WINDOW_SIZE"], config["HORIZON"], features, target, config["GROUP_COLUMN"])

# 3. Crear dataset y dataloader
train_dataset_full = DengueDataset(train_sequences_full, train_labels_full)
train_loader_full = DataLoader(train_dataset_full, batch_size=config["BATCH_SIZE"], shuffle=False)

In [ ]:
# 4. Entrenar con todo el dataset
criterion = nn.MSELoss()
optimizer = optimizer_choices[best_model['best_params']['optimizer']](best_model_trained.parameters(), lr=best_model['best_params']['lr'])

print("\n🔁 Reentrenando con todo el dataset (train + val)...\n")
epochs_retrain = 50
for epoch in range(epochs_retrain):
  best_model_trained.train()
  train_loss = 0
  for x_batch, y_batch in train_loader_full:
    x_batch, y_batch = x_batch.to(DEVICE), y_batch.to(DEVICE)
    optimizer.zero_grad()
    loss = criterion(best_model_trained(x_batch), y_batch)
    loss.backward()
    optimizer.step()
    train_loss += loss.item()
  print(f'Epoch {epoch+1}/{epochs_retrain}, Train Loss: {(train_loss/len(train_loader_full)):.4f}')

In [ ]:
# Preparar dataset de prueba
test_dataset = DengueTestDataset(test_sequences)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
# Generar predicciones
predictions = []
with torch.no_grad():
    for X in test_loader:
        X = X.to(DEVICE)
        output = best_model_trained(X)
        predictions.append(output.cpu().numpy())
predictions = np.concatenate(predictions)
predictions = scaler_target.inverse_transform(predictions.reshape(-1, 1)).flatten()

# Preparar submission
df_submission = pd.DataFrame({'id': ids, 'dengue': predictions})
df_submission.to_csv('submission.csv', index=False)
print(f'Submission guardado en submission.csv, con {len(df_submission)} predicciones.')